In [1]:
!pip install agentai==0.0.4

ERROR: Could not find a version that satisfies the requirement agentai==0.0.4 (from versions: 0.0.1, 0.0.2, 0.0.3)
ERROR: No matching distribution found for agentai==0.0.4


In [2]:
%load_ext autoreload
%autoreload 2

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

from agentai.api import chat_complete, chat_complete_execute_fn
from agentai.annotations import tool, ToolRegistry
from agentai.conversation import Conversation
from pydantic import BaseModel, Field
from typing import List, Optional

extraction_registry = ToolRegistry()  # Namespace for functions

In [3]:
class Medicines(BaseModel):
    name: str = Field(..., description="The name of the medicine")
    dosage: Optional[str] = Field(..., description="The dosage of the medicine")
    frequency: Optional[str] = Field(..., description="The frequency of the medicine")
    effect: Optional[str] = Field(..., description="The response to existing medication")

class Symptoms(BaseModel):
    description: str = Field(..., description="The description of the symptom")
    severity: Optional[str] = Field(..., description="The severity of the symptom", enum=["Acute","Mild", "Moderate", "Severe/Chronic"])
    duration: Optional[str] = Field(..., description="The duration of the symptom",)

class PatientInformation(BaseModel):
    gender: str = Field(..., description="Gender of the patient", enum=["Male", "Female", "Other"])
    age: int = Field(..., description="Age of the patient")
    symptoms: Optional[List[Symptoms]] = Field(..., description="Symptoms that the patient is currently experiencing. Each symptom should be classified into separate item in the list.")
    medicines: Optional[List[Medicines]] = Field(..., description="Medicines that the patient is currently taking. Each medicine should be classified into separate item in the list.")


@tool(extraction_registry)
def save_patient_history(history: PatientInformation) -> None:
    """
    Save the patient's history which consists of gender, age, symptoms and medicines

    Args:
        history (PatientInformation): The patient's history

    Returns:
        str: The current weather
    """
    print(f"Saving patient history: {history}")

In [4]:
convo = Conversation()
convo.add_message(role="system", content="""You are an AI assistant.
- Carefully think about functions to use.
- DO NOT make assumptions.
- Use `null` if you are not confident""",)

convo.add_message(role="user", content=""""49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares. 
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream""")
completion = chat_complete(convo, model=GPT_MODEL, tool_registry=extraction_registry)
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "save_patient_history",
    "arguments": "{\n  \"history\": {\n    \"gender\": \"Male\",\n    \"age\": 49,\n    \"symptoms\": [\n      {\n        \"description\": \"Chronic macular rash to face & hair\",\n        \"severity\": \"Moderate\",\n        \"duration\": \"Unknown\"\n      },\n      {\n        \"description\": \"Itchy\",\n        \"severity\": \"Mild\",\n        \"duration\": \"Unknown\"\n      },\n      {\n        \"description\": \"Flaky, slightly scaly\",\n        \"severity\": \"Mild\",\n        \"duration\": \"Unknown\"\n      }\n    ],\n    \"medicines\": [\n      {\n        \"name\": \"OTC steroid cream\",\n        \"dosage\": \"Unknown\",\n        \"frequency\": \"Unknown\",\n        \"effect\": \"Moderate response\"\n      }\n    ]\n  }\n}"
  }
}


In [5]:
import json
# save_patient_history.from_completion(completion)
# print(json.dumps(save_patient_history._schema, indent=4))
from pydantic import validate_arguments
# val_func = validate_arguments(save_patient_history)
# val_func(json.loads(completion.choices[0].message["function_call"]["arguments"]))
# PatientInformation(**json.loads())
arguments = completion.choices[0].message["function_call"]["arguments"]
history = json.loads(arguments)["history"]
PatientInformation(**history)

PatientInformation(gender='Male', age=49, symptoms=[Symptoms(description='Chronic macular rash to face & hair', severity='Moderate', duration='Unknown'), Symptoms(description='Itchy', severity='Mild', duration='Unknown'), Symptoms(description='Flaky, slightly scaly', severity='Mild', duration='Unknown')], medicines=[Medicines(name='OTC steroid cream', dosage='Unknown', frequency='Unknown', effect='Moderate response')])

In [6]:
save_patient_history._schema

{'name': 'save_patient_history',
 'description': "Save the patient's history which consists of gender, age, symptoms and medicines",
 'parameters': {'type': 'object',
  'properties': {'history': {'title': 'PatientInformation',
    'type': 'object',
    'properties': {'gender': {'title': 'Gender',
      'description': 'Gender of the patient',
      'enum': ['Male', 'Female', 'Other'],
      'type': 'string'},
     'age': {'title': 'Age',
      'description': 'Age of the patient',
      'type': 'integer'},
     'symptoms': {'title': 'Symptoms',
      'description': 'Symptoms that the patient is currently experiencing. Each symptom should be classified into separate item in the list.',
      'type': 'array',
      'items': {'title': 'Symptoms',
       'type': 'object',
       'properties': {'description': {'title': 'Description',
         'description': 'The description of the symptom',
         'type': 'string'},
        'severity': {'title': 'Severity',
         'description': 'The seve